In [2]:
import pandas as pd
import numpy as np
import joblib

In [3]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split


### joblib保存

In [48]:
%%time
# 导入训练数据集
data_date = pd.read_csv('../../preprocess_data/train_x_date.csv').drop(columns=['id','loan_hour'])
data_raw = joblib.load('../../preprocess_data_new/train_ax_nodup.lz4').head(33465).drop(columns=['loan_dt','id','tag'])
data_tag = pd.read_csv('../../preprocess_data/train_x_33465.csv',usecols=['tag'])
major_df = joblib.load('../../preprocess_data_discrete/major_df.lz4').head(33465)
data_null = pd.read_csv('../../preprocess_data_new/train_ax_row_null.csv',nrows=33465).drop(columns=['id'])
maj_cnt_df = joblib.load('../../preprocess_data_discrete/maj_cnt_df.lz4').head(33465)

data = pd.concat([data_date,data_raw,data_tag,major_df,data_null,maj_cnt_df],axis=1)

CPU times: user 43.8 s, sys: 8.37 s, total: 52.2 s
Wall time: 57.5 s


In [53]:
%%time
joblib.dump(data,'./train_data1.lz4',compress='lz4')

CPU times: user 8.78 s, sys: 1.86 s, total: 10.6 s
Wall time: 18.7 s


['./train_data1.lz4']

In [38]:
%%time
# 导入测试数据集

valid_date = pd.read_csv('../../preprocess_data/valid_date.csv').drop(columns=['id','loan_hour'])
valid_raw = joblib.load('../../preprocess_data_new/valid_nodup.lz4').drop(columns=['id','loan_dt'])
valid_tag = pd.read_csv('../predict_tag/tag.csv',usecols=['tag'])
major_test = joblib.load('../../preprocess_data_discrete/major_test.lz4')
valid_null = pd.read_csv('../../preprocess_data_new/valid_row_null.csv').drop(columns=['id'])
maj_cnt_test = joblib.load('../../preprocess_data_discrete/maj_cnt_test.lz4')

valid = pd.concat([valid_date,valid_raw,valid_tag,major_test,valid_null,maj_cnt_test],axis=1)

CPU times: user 7.92 s, sys: 1.36 s, total: 9.28 s
Wall time: 10.4 s


In [40]:
%%time
joblib.dump(valid,'./valid_data1.lz4',compress='lz4')

CPU times: user 5.46 s, sys: 640 ms, total: 6.1 s
Wall time: 6.14 s


['./valid_data1.lz4']

### 导入数据

In [3]:
%%time
# train
data = joblib.load('./train_data1.lz4')

CPU times: user 7.28 s, sys: 2.88 s, total: 10.2 s
Wall time: 12.6 s


In [4]:
data_label = pd.read_csv('../../preprocess_data/train_y_33465.csv',usecols=['label'])
x = data.fillna(-1).values
y = data_label.values.ravel()

In [6]:
%%time
# test
valid = joblib.load('./valid_data1.lz4')
x_test = valid.fillna(-1).values

CPU times: user 6.5 s, sys: 6.99 s, total: 13.5 s
Wall time: 14.7 s


In [15]:
test_id = pd.read_csv('../../preprocess_data_new/valid_date.csv',usecols=['id']).values.ravel()

### 特征组合

In [4]:
%%time
# 导入训练数据集
data_date = pd.read_csv('../../preprocess_data/train_x_date.csv').drop(columns=['id','loan_hour'])
week_df = joblib.load('../../preprocess_data_discrete/week_df.lz4').head(33465)
data_raw = joblib.load('../../preprocess_data_new/train_ax_nodup.lz4').head(33465).drop(columns=['loan_dt','id','tag'])
data_null = pd.read_csv('../../preprocess_data_new/train_ax_row_null.csv',nrows=33465).drop(columns=['id'])
data_tag = pd.read_csv('../../preprocess_data/train_x_33465.csv',usecols=['tag'])

# maj_cnt_df = joblib.load('../../preprocess_data_discrete/maj_cnt_df.lz4').head(33465)
# cat_df = joblib.load('../../preprocess_data_discrete/cat_df.lz4').head(33465)
# discrete_df = joblib.load('../../preprocess_data_discrete/discrete_df.lz4').head(33465)
# scale_df = joblib.load('../../preprocess_data_discrete/scale_df.lz4').head(33465)
# major_df = joblib.load('../../preprocess_data_discrete/major_df.lz4').head(33465)
# day_val_df = joblib.load('../../preprocess_data_discrete/day_val_df.lz4').head(33465)

data_label = pd.read_csv('../../preprocess_data/train_y_33465.csv',usecols=['label'])

CPU times: user 21.5 s, sys: 3.28 s, total: 24.8 s
Wall time: 24.8 s


In [22]:
data_tag.iloc[30464]

tag    0
Name: 30464, dtype: int64

In [6]:
data = pd.concat([data_date,data_raw,data_null,data_tag],axis=1,copy=False)

In [7]:
x = data.fillna(-1).values
y = data_label.values.ravel()
x.shape

(33465, 6702)

In [24]:
%%time
# 导入测试数据集

valid_date = pd.read_csv('../../preprocess_data/valid_date.csv').drop(columns=['id','loan_hour'])
valid_raw = joblib.load('../../preprocess_data_new/valid_nodup.lz4').drop(columns=['id','loan_dt'])
valid_tag = pd.read_csv('../predict_tag/tag.csv',usecols=['tag'])
valid_null = pd.read_csv('../../preprocess_data_new/valid_row_null.csv').drop(columns=['id'])
major_test = joblib.load('../../preprocess_data_discrete/major_test.lz4')
maj_cnt_test = joblib.load('../../preprocess_data_discrete/maj_cnt_test.lz4')

valid = pd.concat([valid_date,valid_raw,valid_tag,valid_null,major_test,maj_cnt_test],axis=1)

CPU times: user 8.8 s, sys: 1.66 s, total: 10.5 s
Wall time: 12.1 s


In [25]:
x_test = valid.fillna(-1).values

In [26]:
x_test.shape

(20000, 11334)

## 数据线上和线下

array([0., 0., 0., ..., 0., 0., 0.])

In [15]:
%%time
# 本地验证
# 1.有标签
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=2018,test_size=0.3,stratify=y)
# 2. 过采样
# from imblearn.over_sampling import BorderlineSMOTE
# x_resampled, y_resampled = BorderlineSMOTE().fit_resample(x_train, y_train)

# 线上测试
# x_train = x
# y_train = y

# valid_date = pd.read_csv('../../preprocess_data/valid_date.csv').drop(columns=['id','loan_hour'])
# # valid_raw = pd.read_csv('../../preprocess_data/valid.csv').drop(columns=['id','loan_dt'])
# valid_raw = pd.read_csv('../../preprocess_data_new/valid_nodup.csv').drop(columns=['id','loan_dt'])
# valid_null = pd.read_csv('../../preprocess_data_new/valid_row_null.csv').drop(columns=['id'])
# valid_tag = pd.read_csv('../predict_tag/tag.csv',usecols=['tag'])
# valid = pd.concat([valid_date,valid_raw,valid_null,valid_tag],axis=1)
# test_id = pd.read_csv('../../preprocess_data_new/valid_date.csv',usecols=['id']).values.ravel()
# x_test = valid.fillna(-1).values

CPU times: user 2.08 s, sys: 1.03 s, total: 3.1 s
Wall time: 3.1 s


In [16]:
x_train.shape

(23425, 6702)

## 本地

In [17]:
%%time
#2.建立模型 
model_names = ['rfc','gbc','xgb','lgb']
model_name = model_names[2]
if model_name == 'rfc':
    # RandomForestClassifier
    from sklearn.ensemble import RandomForestClassifier
    rfc  =RandomForestClassifier(n_estimators=2500,n_jobs =36,max_features='sqrt',class_weight='balanced',verbose =1,random_state=2018)
elif model_name == 'gbc':
    from sklearn.ensemble import GradientBoostingClassifier
    gbc = GradientBoostingClassifier(loss='deviance',learning_rate =0.1,n_estimators=300,subsample=0.9,max_depth=3,verbose=1,random_state=2018)
elif model_name == 'xgb':
    import xgboost as xgb
    params={
    'booster':'gbtree',
    'objective': 'binary:logistic',
    'early_stopping_rounds':100,
    'scale_pos_weight': float(len(data_label)-np.sum(data_label.values))/float(np.sum(data_label.values)),  # 负例样本除以正例样本
    'eval_metric': 'auc',
    'gamma':1,
    'max_depth':6,
    'lambda':1,
    'subsample':0.9,
    'colsample_bytree':0.9,
    'min_child_weight':1, 
    'eta': 0.04,
    'seed':2010,
    'nthread':32
        }
elif model_name == 'lgb':
    import lightgbm as lgb
    parameters = {
        'boost':'gbdt',
        'num_leaves':135, 
        'scale_pos_weight':float(len(y_train)-np.sum(y_train.ravel()))/float(np.sum(y_train.ravel())),
        'max_depth':-1,
        'learning_rate':.05,
        'max_bin':200,
        'min_data_in_leaf' : 60,
        'objective':'binary',
        'metric':'auc',
        'verbose':1,
        'num_threads':36
    }
       
else:
    pass

#3.训练模型
print('开始训练！')
import joblib
if model_name == 'rfc':
    rfc.fit(x_train,y_train)
#     joblib.dump(rfc,'./model/rfc_model')
elif model_name == 'gbc':
    gbc.fit(x_train,y_train)
#     joblib.dump(gbc,'./model/rgbc_model')
elif model_name =='xgb':
    dtrain = xgb.DMatrix(x_train,y_train)
    dvalid = xgb.DMatrix(x_test,y_test)
    dtest = xgb.DMatrix(x_test)
    watchlist  = [(dvalid,'valid')]
    xgb_model = xgb.train(params,dtrain,num_boost_round=300,evals=watchlist)
#     xgb_model.save_model('./model/xgb_model')
elif model_name == 'lgb':
    lgb_train = lgb.Dataset(x_train, y_train.ravel())
    lgb_model = lgb.train(parameters,lgb_train,num_boost_round=300)
#     lgb_model.save_model('./model/lgb_model')
else:
    pass
# lg.fit(x_train,y_train.ravel())
# lsvm.fit(x_train,y_train)
# history = model.fit(x_train,y_train,epochs=2,batch_size=1024,class_weight = 'auto',validation_data=(x_test,y_test))

开始训练！
[0]	valid-auc:0.721024
[1]	valid-auc:0.743645
[2]	valid-auc:0.757624
[3]	valid-auc:0.76535
[4]	valid-auc:0.76947
[5]	valid-auc:0.771044
[6]	valid-auc:0.772436
[7]	valid-auc:0.774228
[8]	valid-auc:0.778648
[9]	valid-auc:0.780205
[10]	valid-auc:0.782302
[11]	valid-auc:0.783317
[12]	valid-auc:0.783445
[13]	valid-auc:0.785957
[14]	valid-auc:0.786131
[15]	valid-auc:0.786251
[16]	valid-auc:0.787698
[17]	valid-auc:0.788291
[18]	valid-auc:0.788792
[19]	valid-auc:0.790014
[20]	valid-auc:0.790437
[21]	valid-auc:0.79025
[22]	valid-auc:0.790258
[23]	valid-auc:0.790956
[24]	valid-auc:0.791717
[25]	valid-auc:0.792507
[26]	valid-auc:0.793169
[27]	valid-auc:0.793711
[28]	valid-auc:0.794321
[29]	valid-auc:0.795006
[30]	valid-auc:0.795449
[31]	valid-auc:0.795942
[32]	valid-auc:0.796105
[33]	valid-auc:0.796183
[34]	valid-auc:0.797065
[35]	valid-auc:0.797853
[36]	valid-auc:0.798445
[37]	valid-auc:0.798718
[38]	valid-auc:0.799216
[39]	valid-auc:0.800073
[40]	valid-auc:0.800651
[41]	valid-auc:0.801379

In [50]:
#4.预测结果
print('开始预测！')
if model_name=='rfc':
    y_pre = rfc.predict_proba(x_test)[:,1]
elif model_name=='gbc':
    y_pre = gbc.predict_proba(x_test)[:,1]
elif model_name=='xgb':
    dtest = xgb.DMatrix(x_test)
    y_pre = xgb_model.predict(dtest).ravel()
elif model_name=='lgb':
    y_pre = lgb_model.predict(x_test)
else:
    pass

# y_pre = lsvm.decision_function(X_test)
# y_pre = lg.predict(X_test)
from sklearn import metrics
auc = metrics.roc_auc_score(y_test, y_pre)
print('AUC:',auc)

开始预测！
AUC: 0.8307681067058688


## 未作stratify
nodup + null + tag +(fillna(-1))AUC: 0.82785  
nodup + null + tag +(fillna(-1)) + maj_cnt AUC:    
nodup + null + tag +(fillna(-1)) + major_df + maj_cnt_df  AUC: 0.832029(200)
nodup + null + tag +(fillna(-1)) + major_df AUC:0.829731  
nodup + null + tag +(fillna(-1)) + major_df + maj_cnt_df + discrete_df:0.8301
data_raw被替换 AUC：0.821537  
nodup + null + tag +(fillna(-1)) + major_df + maj_cnt_df + week_df AUC: 0.826666  
nodup + null + tag +(fillna(-1))  + maj_cnt_df + week_df AUC:0.828063  
nodup + null + tag +(fillna(-1))  + maj_cnt_df + maj_cnt_df + day_val_df AUC: 0.828095

## 线上

### xgb bagging

In [36]:
test_id = pd.read_csv('../../preprocess_data_new/valid_date.csv',usecols=['id']).values.ravel()

In [37]:
import xgboost as xgb
dtrain = xgb.DMatrix(x, y, feature_names=list(data.columns))
dtest = xgb.DMatrix(x_test, feature_names=list(valid.columns))

In [38]:
def pipeline(dtrain,dtest,test_id,iteration,random_seed,gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight):
    if max_depth==6:
        num_boost_round = 210
    elif max_depth==7:
        num_boost_round = 180
    elif max_depth==8:
        num_boost_round = 150
    
    params={
    'booster':'gbtree',
    'objective': 'binary:logistic',
    'early_stopping_rounds':100,
    'scale_pos_weight': float(len(data_label)-np.sum(data_label.values))/float(np.sum(data_label.values)),  # 负例样本除以正例样本
    'eval_metric': 'auc',
    'gamma':gamma,
    'max_depth':max_depth,
    'lambda':lambd,
    'subsample':subsample,
    'colsample_bytree':colsample_bytree,
    'min_child_weight':min_child_weight, 
    'eta': 0.04,
    'seed':random_seed,
    'nthread':32
        }
    watchlist  = [(dtrain,'train')]
    model = xgb.train(params,dtrain,num_boost_round=num_boost_round,evals=watchlist)
    model.save_model('./xgb_models1/xgb{0}.model'.format(iteration))
    
    #predict test set
    test_y = model.predict(dtest)
    test_result = pd.DataFrame(test_id,columns=["id"])
    test_result['score'] = test_y
    test_result.to_csv("./xgb_preds1/xgb{0}.csv".format(iteration),index=None,encoding='utf-8')
    
    #get feature score
    feature_score = model.get_fscore()
    feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
    fs = []
    for (key,value) in feature_score:
        fs.append("{0},{1}\n".format(key,value))
    
    with open('./xgb_featurescores1/feature_score_{0}.csv'.format(iteration),'w') as f:
        f.writelines("feature,score\n")
        f.writelines(fs)

In [40]:
%%time
import sys,random

random_seed = list(range(1000,2000,20))
gamma = [i/1000.0 for i in range(100,200,2)]
max_depth = [6,7,8]
lambd = list(range(200,400,2))
subsample = [i/1000.0 for i in range(600,700,2)]
colsample_bytree = [i/1000.0 for i in range(250,350,2)]
min_child_weight = [i/1000.0 for i in range(200,300,2)]
random.shuffle(random_seed)
random.shuffle(gamma)
random.shuffle(max_depth)
random.shuffle(lambd)
random.shuffle(subsample)
random.shuffle(colsample_bytree)
random.shuffle(min_child_weight)

with open('params.pkl','wb') as f:
    joblib.dump((random_seed,gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight),f)

#to reproduce my result, uncomment following lines
"""
with open('params_for_reproducing.pkl','rb') as f:
    random_seed,gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight = pickle.load(f)    
"""

for i in range(36):
    pipeline(dtrain,dtest,test_id,i,random_seed[i],gamma[i],max_depth[i%3],lambd[i],subsample[i],colsample_bytree[i],min_child_weight[i])

[0]	train-auc:0.794805
[1]	train-auc:0.822588
[2]	train-auc:0.841422
[3]	train-auc:0.850825
[4]	train-auc:0.855166
[5]	train-auc:0.859018
[6]	train-auc:0.861039
[7]	train-auc:0.863852
[8]	train-auc:0.86654
[9]	train-auc:0.867068
[10]	train-auc:0.86986
[11]	train-auc:0.872057
[12]	train-auc:0.874137
[13]	train-auc:0.875415
[14]	train-auc:0.876608
[15]	train-auc:0.879011
[16]	train-auc:0.880089
[17]	train-auc:0.881867
[18]	train-auc:0.8834
[19]	train-auc:0.885735
[20]	train-auc:0.886665
[21]	train-auc:0.888256
[22]	train-auc:0.890089
[23]	train-auc:0.891641
[24]	train-auc:0.892029
[25]	train-auc:0.893119
[26]	train-auc:0.894244
[27]	train-auc:0.895733
[28]	train-auc:0.896812
[29]	train-auc:0.897417
[30]	train-auc:0.898262
[31]	train-auc:0.899204
[32]	train-auc:0.900114
[33]	train-auc:0.900881
[34]	train-auc:0.90234
[35]	train-auc:0.903354
[36]	train-auc:0.903963
[37]	train-auc:0.904745
[38]	train-auc:0.905866
[39]	train-auc:0.906573
[40]	train-auc:0.907137
[41]	train-auc:0.907834
[42]	tr